In [1]:
### IMPORTING LIBRARIES
import bs4
import pandas as pd
import requests
import re
import string
import telegram

In [37]:
### GETTING LINK AND CREATING BS4 OBJECT
res = requests.get("http://www.dailytenminutes.com/p/short-stories.html")
res.raise_for_status() # checking if request was successful
readsSoup = bs4.BeautifulSoup(res.text,'html.parser')

In [39]:
### GETTING ALL LINKS
links = []
for a in readsSoup.find_all('a',href=True): #Finding all links from bs4 object
    if a.text:
        links.append(a['href'])
l = len(links)
#print(l)

### USING REGEX TO FIND LINKS FOR STORIES
def cleaningLinks(L):
    rlinks = []
    for i in links:
        linkreg = re.search('[a-zA-Z.]+/\d+/\d+',i)
        if linkreg!=None:
            rlinks.append(i)
    return rlinks

a = cleaningLinks(links)

### STORING LINKS TO CSV WITH 'DONE' COLUMN WITH DEFAULT VALUE 0
def csvLinks(links):
    df = pd.DataFrame(links,columns=["Links"])
    df["Done"] = 0
    df.to_csv("links.csv",index = False)
csvLinks(a)



In [40]:
### EXTRACTING LINK FROM CSV AND MARKING IT DONE 
def getLink():
    df = pd.read_csv("links.csv")
    for index,row in df.iterrows():
        if row['Done'] == 1:
            continue
        else:
            print(index,row['Links'],row['Done'])
            df.loc[index,"Done"] = 1
            df.to_csv("links.csv", index=False)
            return row['Links']
            break
a = getLink()
#print(a)

0 http://www.dailytenminutes.com/2017/04/in-class-i-teach-for-adults-i-recently.html 0


In [41]:
### MINE THE STORY FROM EXTRACTED LINK
bookres = requests.get(a)
res.raise_for_status()
linksSoup = bs4.BeautifulSoup(bookres.text,'html.parser')
L = []
def getStory(): 
    p = linksSoup.findAll('article')
    for r in p:
        L.append(r.get_text())
getStory()


In [42]:
### CLEANING THE STORY
def cleanStory(L):
    f = re.sub(r'\\n+|\\| Read 300+ stories on DailyTenMinutes.com'," ",L[0]) #removing all \n
    #re.sub(r' Read 300+ stories on DailyTenMinutes.com '," ",p)
    f1 = " ".join(f.split())
    flag = True
    while flag == True:    # removing the intro part
        for i in range(len(f1)//2):
            if f1[i] == "." and f1[i-1] == "." and f1[i-2] == ".": #Pattern
                p = f1[i+1:len(f1)]
                flag = False
    return p
a = cleanStory(L)
print(type(a))

<class 'str'>


In [43]:
my_token = '1609959829:AAGB6Uvtbhv82EOM2ceevKcl5befJSWeIQI'

def send(msg, chat_id, token=my_token):
    bot = telegram.Bot(token=token)
    bot.sendMessage(chat_id=chat_id, text=msg)
send(a,1211116531,my_token)